In [1]:
# none of these values are real, they are just filler values
k_concrete = 1
t_concrete = 1
k_casing = 0.5
t_casing = 1
k_tube = .7
t_tube = 1

def conduction_resistance(k, t, a):
    return t/(k*a)

Because of conservation of energy, the energy balance for a control volume can be written as:
$$
Q_{in}  = \dot m \big[ (h_2 - h_1) + \frac{1}{2}(v_2^2 - v_1^2) + g(z_2-z_1) + g h_l \big] 
$$

In [2]:
class State:
    def __init__(self, h, v, z):
        self.h = h
        self.v = v
        self.z = z

In [3]:
def heat_transfer(state2,state1,mdot, head_loss):
    qin = mdot * ((state2.h - state1.h) + 0.5 * (state2.v**2 - state1.v**2) + 9.81 * (state2.z - state1.z) + g * head_loss)
    return qin